In [1]:
from bs4 import BeautifulSoup
import urllib
import nltk
import re
from pprint import pprint
import csv
from collections import OrderedDict
import pandas as pd
import numpy as np
import unicodedata
import got_tools as gt
from __future__ import print_function

In [2]:
# load in character list
character_list = []
char_list = open('data/char_list_s5.txt', 'r')
for char in char_list:  
    character_list.append(char.replace('\n',''))
    
char_list.close()

In [4]:
# this code block is useful for testing if it worked on an individual character
char = 'Arya_Stark'
char = char.replace('\n','') # remove newline sometimes contained in character name


# initialize some vars
c = {'Season 1': {},
    'Season 2': {},
    'Season 3': {},
    'Season 4': {},
    'Season 5': {},
    'Background': {}}
key_words = ["Season(s)", "First seen", "Last seen", "Appeared in", "Mentioned in", "Titles", "Also known as", "Status",
             "Age", "Date of birth", "Death", "Origin", "Allegiance", "Culture", "Religion", "Family", "Portrayed by"]
char_info = {}
# initialize dicionary with key words
for key in key_words:
    char_info[key] = {}
char_info['Name'] = {} # since we handle name separately, we decide not to include it in the key_words list

# get the data from the wiki page
url = 'http://gameofthrones.wikia.com/wiki/' + char
page = urllib.urlopen(url)
soup = BeautifulSoup(page.read(), "html.parser")


# Output structured info from the right side of the wiki page
print('Name: {}'.format(char.replace('_',' ')))

for word in key_words:
    keys = soup.find(text=word)
    if keys is not None:
        value = keys.parent.findNextSibling('div')
        if value is not None:
            char_info[word][char] = value.text
            print('{}: {}'.format(word, value.text))
        #TODO: special case for family, create a dictionary


# call function to get text data
char_c = gt.return_character_dictionary(soup, char, c)

for item in char_c:
    if char_c[item] is not None:
        print('{} text: found!'.format(item))
    else:
        print('{} text: absent!'.format(item))

Name: Arya Stark
Season(s): 1, 2, 3, 4, 5, 6
First seen: "Winter is Coming"
Appeared in: 39 episodes (see below)
Titles: Princess
Also known as: ArryLanna
Status: Alive
Age: 16 in Season 6[1]
Origin: Winterfell
Allegiance: House StarkFaceless Men
Culture: Northmen
Religion: Old Gods of the Forest (formerly)Faith of the Seven (formerly)Many-Faced God
Family: {Eddard Stark} - father{Catelyn Stark} - mother{Robb Stark} - brotherSansa Stark - sisterBran Stark - brotherRickon Stark - brother{Jon Snow} - bastard paternal half-brotherBenjen Stark - paternal uncle{Brandon Stark} - paternal uncle{Lyanna Stark} - paternal aunt{Jon Arryn} - uncle by marriagePetyr Baelish - uncle by marriageRoslin Frey - aunt by marriage{Rickard Stark} -  paternal grandfatherBrynden Tully - maternal great-uncle{Hoster Tully} - maternal grandfather{Minisa Tully} - maternal grandmotherEdmure Tully - maternal uncle{Lysa Arryn} - maternal auntRobin Arryn - maternal first cousin{Talisa Maegyr} - sister-in-lawTyrion Lan

Now we will loop through all the characters in character list and pull:

* semi-structured data from the right side of their wiki page (detailed by a number of key words)
* unstructured text data from the background/season sections

In [5]:
# now using the above function, create a dictionary with all our characters and their text data
c = {'Season 1': {},
    'Season 2': {},
    'Season 3': {},
    'Season 4': {},
    'Season 5': {},
    'Background': {}}

# at the same time get other structured info
key_words = ["Season(s)", "First seen", "Last seen", "Appeared in", "Mentioned in", "Titles", "Also known as", "Status",
             "Age", "Date of birth", "Death", "Origin", "Allegiance", "Culture", "Religion", "Family", "Portrayed by"]

info = {}
# initialize dicionary with key words
for key in key_words:
    info[key] = {}
info['Name'] = {} # since we handle name separately, we decide not to include it in the key_words list
count_char = 1

for char in character_list:
    char = char.replace('\n','') # remove newline sometimes contained in character name
    
    if np.mod( character_list.index(char), 5 ) == 0:
        print('') #newline, i.e. only print 5 character names per line
    print ('{} .. '.format(char), end="")
        
    # get the data from the wiki page
    url = 'http://gameofthrones.wikia.com/wiki/' + char
    page = urllib.urlopen(url)
    soup = BeautifulSoup(page.read(), "html.parser")
    
    # Output structured info from the right side of the wiki page
    info['Name'][char] = char.replace('_',' ')

    for word in key_words:
        keys = soup.find(text=word)
        if keys is not None:
            value = keys.parent.findNextSibling('div')
            if value is not None:
                info[word][char] = value.text

            #TODO: special case for family, create a dictionary
    
    
    # call function to get text data
    c = gt.return_character_dictionary(soup, char, c)


Aeron_Greyjoy .. Areo_Hotah .. Arianne_Martel .. Arya_Stark .. Balon_Greyjoy .. 
Barristan_Selmy .. Ben_Plumm .. Beric_Dondarrion .. Bowen_Marsh .. Brienne_of_Tarth .. 
Bronn .. Brynden_Tully .. Cersei_Lannister .. Daario_Naharis .. Daenerys_Targaryen .. 
Davos_Seaworth .. Doran_Martell .. Ellaria_Sand .. Euron_Greyjoy .. Gendry .. 
Gerold_Dayne_The_Darkstar .. Pycelle .. Gregor_Clegane .. Grey_Worm .. Griff .. 
Hizdahr_zo_Loraq .. Jaime_Lannister .. Janos_Slynt .. Jaqen_H'ghar .. Jon_Snow .. 
Jorah_Mormont .. Kevan_Lannister .. Lady_Stoneheart .. Lancel_Lannister .. Loras_Tyrell .. 
Lord_Varys .. Mace_Tyrell .. Maester_Aemon .. Mance_Rayder .. Margaery_Tyrell .. 
Melisandre .. Meryn_Trant .. Missandei .. Myrcella_Baratheon .. Nymeria_Sand .. 
Obara_Sand .. Olenna_Tyrell .. Olly .. Petyr_Baelish .. Podrick_Payne .. 
Qyburn .. Ramsay_Bolton .. Rattleshirt .. Reznak_mo_Reznak .. Robin_Arryn .. 
Roose_Bolton .. Samwell_Tarly .. Sansa_Stark .. Ser_Arys_Oakheart .. Stannis_Baratheon .. 
Hi

In [6]:
# create data frames of the above
df_info_unicode = pd.DataFrame.from_dict(info)
df_txt_unicode = pd.DataFrame.from_dict(c, dtype='unicode')

In [7]:
# convert the unicode dataframe to ascii text - as best we can, this will inevitably remove/alter some characters
def convert_text(s):
    if str(type(s)) == "<type 'unicode'>":
        txt = unicodedata.normalize('NFKD', s).encode('ascii','ignore')
        return txt.replace("\\'", "'")  # remove escape characters
    else:
        return None

d = {}
for col in df_info_unicode.columns:
    d[col] = df_info_unicode[col].apply(convert_text)

df_info = pd.DataFrame(d)

d = {}
for col in df_txt_unicode.columns:
    d[col] = df_txt_unicode[col].apply(convert_text)

df_txt = pd.DataFrame(d)

In [10]:
# load in the scores from season 5
scores = pd.read_csv('data/season5_scores.csv')
scores.head()

,Killing,SexNudity,Insult,Drinking,Injury,Total
Name,,,,,,
Aeron_Greyjoy,0,0,0,0,0,0
Areo_Hotah,0,0,7,0,4,11
Arianne_Martel,0,0,0,0,0,0
Arya_Stark,18,0,28,0,40,86
Balon_Greyjoy,0,0,0,0,0,0


In [11]:
# write data to file
df = scores
df = df.merge(df_info, left_index=True, right_index=True, how='left')
df = df.merge(df_txt, left_index=True, right_index=True, how='left')
df.head()

df.to_csv('data/got_data.csv')

The family field in the semi-structured data could be better parsed ... kind of made a start here but didn't get very far.


```python
# try to parse family
value = info['Family']
tmp = value.split('{')
tmp_dict = {} 
for tmpval in tmp:
    if tmpval == '' or tmpval is None:
        continue
    
    if ' - ' in tmpval:
        tmp_dict[ tmpval.split(' - ')[1] ] = tmpval.split('}')[0]
        print('{}     tmp_dict[{}] = {}').format(tmpval,  tmpval.split(' - ')[1] , tmpval.split('}')[0])
```